In [1]:
import pandas as pd
from matplotlib import pyplot
import re
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [2]:
def read_pgm(filename, byteorder='>'):
    """Return image data from a raw PGM file as numpy array.
    Format specification: http://netpbm.sourceforge.net/doc/pgm.html
    """
    with open(filename, 'rb') as f:
        buffer = f.read()
    try:
        header, width, height, maxval = re.search(
            b"(^P5\s(?:\s*#.*[\r\n])*"
            b"(\d+)\s(?:\s*#.*[\r\n])*"
            b"(\d+)\s(?:\s*#.*[\r\n])*"
            b"(\d+)\s(?:\s*#.*[\r\n]\s)*)", buffer).groups()
    except AttributeError:
        raise ValueError("Not a raw PGM file: '%s'" % filename)
    return np.frombuffer(buffer,
                            dtype='u1' if int(maxval) < 256 else byteorder+'u2',
                            count=int(width)*int(height),
                            offset=len(header)
                            ).reshape((int(height), int(width)))
def view_img(pdata):
    pyplot.imshow(pdata, pyplot.cm.gray)
    pyplot.show()

In [3]:
threshold = 0;
def translate_output(prediction):
    answer_whole = []
    for pred in prediction:
        answer = []
        # background classification
        answer.append(onehot_bg.keys()[np.argmax(pred[0:3])])
        # classification of mass
        b = np.argmax(pred[3:10])
        answer.append(onehot_class.keys()[b])
        
        # severity of mass
        c = np.argmax(pred[10:12])
        answer.append(onehot_severity.keys()[c])
        # x, y , radius of mass
        # answer.append(prediction[12:]*1024.)
        answer_whole.append(answer)
    return answer_whole

In [4]:
with open('D:\dmarq\Projects\MIAS_Mammography\Info.txt') as file:
    data = file.read()
    arr = data.split("\n")
    for i, each in enumerate(arr):
        arr[i] = each.split()
keys = {key:[] for key in arr[0]}
mam_df = pd.DataFrame(data=arr[1:],columns=keys)

In [22]:
mam_df.head(10)

,REFNUM,BG,CLASS,SEVERITY,X,Y,RADIUS
0,mdb001,G,CIRC,B,535,425,197
1,mdb002,G,CIRC,B,522,280,69
2,mdb003,D,NORM,None,None,None,None
3,mdb004,D,NORM,None,None,None,None
4,mdb005,F,CIRC,B,477,133,30
5,mdb005,F,CIRC,B,500,168,26
6,mdb006,F,NORM,None,None,None,None
7,mdb007,G,NORM,None,None,None,None
8,mdb008,G,NORM,None,None,None,None
9,mdb009,F,NORM,None,None,None,None


In [6]:
for key in mam_df["CLASS"].unique():
    print(key,mam_df[mam_df['CLASS'] == key].count(),'\n')

CIRC REFNUM      25
BG          25
CLASS       25
SEVERITY    25
X           24
Y           24
RADIUS      24
dtype: int64 

NORM REFNUM      207
BG          207
CLASS       207
SEVERITY      0
X             0
Y             0
RADIUS        0
dtype: int64 

MISC REFNUM      15
BG          15
CLASS       15
SEVERITY    15
X           15
Y           15
RADIUS      15
dtype: int64 

ASYM REFNUM      15
BG          15
CLASS       15
SEVERITY    15
X           15
Y           15
RADIUS      15
dtype: int64 

ARCH REFNUM      19
BG          19
CLASS       19
SEVERITY    19
X           19
Y           19
RADIUS      19
dtype: int64 

SPIC REFNUM      19
BG          19
CLASS       19
SEVERITY    19
X           19
Y           19
RADIUS      19
dtype: int64 

CALC REFNUM      30
BG          30
CLASS       30
SEVERITY    30
X           27
Y           27
RADIUS      27
dtype: int64 



In [7]:
mam_df = mam_df.drop(mam_df[mam_df["CLASS"] == 'NORM'].iloc[25:].index)

In [8]:
mam_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 148 entries, 0 to 322
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   REFNUM    148 non-null    object
 1   BG        148 non-null    object
 2   CLASS     148 non-null    object
 3   SEVERITY  123 non-null    object
 4   X         119 non-null    object
 5   Y         119 non-null    object
 6   RADIUS    119 non-null    object
dtypes: object(7)
memory usage: 9.2+ KB


In [9]:
#img_data = [] # ref name and pdata
input_list = [] # just pdata
for file_name in mam_df['REFNUM']:
    pdata = read_pgm('D:\dmarq\Projects\MIAS_Mammography\data\{}.pgm'.format(file_name), byteorder='<')
    #img_data.append([file_name,pdata])
    input_list.append(pdata/255.) 
input_list = np.array(input_list)

In [10]:
#output = []
#for img in input_list:
#    arr = []
#    for row in img:
#        arr.append([[e] for e in row])
#    output.append(arr)

In [11]:
#view_img(img_data[69][1])

In [12]:
#print("X mean:",mam_df[mam_df['X'].notnull()]['X'].astype('int').mean())
#print("Y mean:",mam_df[mam_df['Y'].notnull()]['Y'].astype('int').mean())
#print("Radius mean:",mam_df[mam_df['RADIUS'].notnull()]['RADIUS'].astype('int').mean()) # the only real meaningful stat

In [13]:
# encode outputs for each image
onehot_bg = pd.get_dummies(mam_df['BG'],prefix='BG')
onehot_class = pd.get_dummies(mam_df['CLASS'],prefix='CLASS')
onehot_severity = pd.get_dummies(mam_df['SEVERITY'],prefix='SEVERITY')
coords = mam_df[["X","Y","RADIUS"]]
# 
coords_encoded = []
for each in coords.iloc:
    try:
        coords_encoded.append(each.values.astype('int')/1024.)
    except:
        coords_encoded.append(np.zeros(3))
onehot_bg
onehot_class
#onehot_severity

,CLASS_ARCH,CLASS_ASYM,CLASS_CALC,CLASS_CIRC,CLASS_MISC,CLASS_NORM,CLASS_SPIC
0,0,0,0,1,0,0,0
1,0,0,0,1,0,0,0
2,0,0,0,0,0,1,0
3,0,0,0,0,0,1,0
4,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...
281,0,0,0,0,1,0,0
297,0,0,0,1,0,0,0
319,0,0,0,0,1,0,0
321,0,0,0,0,1,0,0


In [14]:
outputs = [] # this formats the output as bg, class, severity,
for i in range(mam_df.shape[0]):
    outputs.append(np.concatenate((onehot_bg.iloc[i].values,onehot_class.iloc[i].values,onehot_severity.iloc[i].values))) # ,coords_encoded[i]
outputs = np.array(outputs)
outputs[0]

array([0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0], dtype=uint8)

In [15]:
input_list = input_list.reshape(mam_df.shape[0], 1024, 1024, 1)

In [16]:
input_shape = (mam_df.shape[0],1024,1024,1)

In [17]:
model = models.Sequential()
model.add(layers.Conv2D(32, 3, activation='relu', input_shape=input_shape[1:]))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(2))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(2))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(2))
model.add(layers.Conv2D(16, 3, activation="relu"))
model.add(layers.MaxPooling2D(2))
model.add(layers.GlobalMaxPooling2D())
model.add(layers.Dense(12, activation="sigmoid"))
model.compile(loss='mse',
                  optimizer=keras.optimizers.Adam(learning_rate=0.0003),
                  metrics=[keras.metrics.AUC()])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 1022, 1022, 32)    320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 1020, 1020, 32)    9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 510, 510, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 508, 508, 32)      9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 506, 506, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 253, 253, 32)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 251, 251, 32)      9

In [18]:
with tf.device('/GPU:0'):
    model.fit(input_list[0:120],outputs[0:120],verbose=1,epochs=25,batch_size=5,shuffle=True)

Epoch 1/25
24/24 [==============================] - 143s 6s/step - loss: 0.2383 - auc: 0.5656
Epoch 2/25
24/24 [==============================] - 144s 6s/step - loss: 0.2000 - auc: 0.5936
Epoch 3/25
24/24 [==============================] - 143s 6s/step - loss: 0.1899 - auc: 0.6106
Epoch 4/25
24/24 [==============================] - 143s 6s/step - loss: 0.1844 - auc: 0.6156
Epoch 5/25
24/24 [==============================] - 143s 6s/step - loss: 0.1805 - auc: 0.6226
Epoch 6/25
24/24 [==============================] - 144s 6s/step - loss: 0.1781 - auc: 0.6249
Epoch 7/25
24/24 [==============================] - 143s 6s/step - loss: 0.1752 - auc: 0.6397
Epoch 8/25
24/24 [==============================] - 143s 6s/step - loss: 0.1733 - auc: 0.6427
Epoch 9/25
24/24 [==============================] - 143s 6s/step - loss: 0.1721 - auc: 0.6439
Epoch 10/25
24/24 [==============================] - 143s 6s/step - loss: 0.1711 - auc: 0.6560
Epoch 11/25
24/24 [==============================] - 143s 6

In [ ]:
y = model.predict(input_list[0:100])

In [24]:
translate_output(y)

[['BG_D', 'CLASS_ARCH', 'SEVERITY_B'],
 ['BG_D', 'CLASS_NORM', 'SEVERITY_B'],
 ['BG_D', 'CLASS_NORM', 'SEVERITY_B'],
 ['BG_D', 'CLASS_NORM', 'SEVERITY_B'],
 ['BG_D', 'CLASS_CIRC', 'SEVERITY_B'],
 ['BG_D', 'CLASS_CIRC', 'SEVERITY_B'],
 ['BG_D', 'CLASS_NORM', 'SEVERITY_B'],
 ['BG_G', 'CLASS_NORM', 'SEVERITY_B'],
 ['BG_D', 'CLASS_NORM', 'SEVERITY_B'],
 ['BG_G', 'CLASS_NORM', 'SEVERITY_B']]

In [1]:
model.evaluate(input_list[120:122],outputs[120:122])

NameError: name 'model' is not defined

In [25]:
model.save('modeltest.h5')